In [ ]:
#| default_exp core

# Source

- Source code for FastSQL, a MiniDataAPI spec implementation for SQLAlchemy V2

In [ ]:
#| export
from dataclasses import dataclass,is_dataclass,asdict,MISSING,fields
import sqlalchemy as sa
from sqlalchemy.orm import Session
from fastcore.utils import *
from fastcore.test import test_fail
from itertools import starmap

## `Database` and `DBTable`

We create a `Database` class and a `DBTable` class (which is returned by `Database.create`), using sqlalchemy v2. These classes will allow us to directly work with dataclasses such as these:

In [ ]:
class User: name:str; pwd:str
class Todo: title:str; name:str; id:int; done:bool=False; details:str=''

In [ ]:
#| export
class Database:
    "A connection to a SQLAlchemy database"
    def __init__(self, conn_str):
        self.conn_str = conn_str
        self.engine = sa.create_engine(conn_str)
        self.meta = sa.MetaData()
        self.meta.reflect(bind=self.engine)
        self.meta.bind = self.engine
        self.conn = self.engine.connect()
        self.meta.conn = self.conn
    
    def execute(self, st, params=None, opts=None): return self.conn.execute(st, params, execution_options=opts)

    def __repr__(self): return f"Database({self.conn_str})"

In [ ]:
db = Database("sqlite:///:memory:")

In [ ]:
#| export
class DBTable:
    "A connection to a SQLAlchemy table, created if needed"
    def __init__(self, table: sa.Table, database: Database, cls):
        self.table,self.db,self.cls = table,database,cls
        table.create(self.db.engine, checkfirst=True)

    def __repr__(self) -> str: return self.table.name
    
    @property
    def t(self)->tuple: return self.table,self.table.c
    
    @property
    def conn(self): return self.db.conn

In [ ]:
#| export
_type_map = {int: sa.Integer, str: sa.String, bool: sa.Boolean}
def _column(name, typ, primary=False): return sa.Column(name, _type_map[typ], primary_key=primary)

In [ ]:
#| export
@patch
def create(self:Database, cls:type, pk='id', name:str|None=None):
    "Get a table object, creating in DB if needed"
    pk = listify(pk)
    mk_dataclass(cls)
    if name is None: name = camel2snake(cls.__name__)
    cols = [_column(o.name, o.type, primary=o.name in pk) for o in fields(cls)]
    tbl = sa.Table(name, self.meta, *cols, extend_existing=True)
    return DBTable(tbl, self, cls)

In [ ]:
users = db.create(User, pk='name')
todos = db.create(Todo, pk='id')

In [ ]:
#| export
@patch
def schema(self:Database):
    "Show all tables and columns"
    inspector = sa.inspect(self.engine)
    res = ''
    for table_name in inspector.get_table_names():
        res += f"Table: {table_name}\n"
        pk_cols = inspector.get_pk_constraint(table_name)['constrained_columns']
        for column in inspector.get_columns(table_name):
            pk_marker = '*' if column['name'] in pk_cols else '-'
            res += f"  {pk_marker} {column['name']}: {column['type']}\n"
    return res

In [ ]:
print(db.schema())

Table: todo
  - title: VARCHAR
  - name: VARCHAR
  * id: INTEGER
  - done: BOOLEAN
  - details: VARCHAR
Table: user
  * name: VARCHAR
  - pwd: VARCHAR



In [ ]:
#| export
@patch
def exists(self:DBTable):
    "Check if this table exists in the DB"
    return sa.inspect(self.db.engine).has_table(self.table.name)

In [ ]:
users.exists()

True

In [ ]:
u0 = User('jph','foo')
u1 = User('rlt','bar')
t0 = Todo('do it', 'jph')

In [ ]:
#| export
def _wanted(obj): return {k:v for k,v in asdict(obj).items() if v not in (None,MISSING)}

In [ ]:
#| export
@patch
def insert(self:DBTable, obj):
    "Insert an object into this table, and return it"
    result = self.conn.execute(sa.insert(self.table).values(**_wanted(obj)).returning(*self.table.columns))
    row = result.one()  # Consume the result set
    self.conn.commit()
    return self.cls(**row._asdict())

In [ ]:
t = todos.insert(t0)
assert t.id
t

Todo(title='do it', name='jph', id=1, done=False, details='')

In [ ]:
u = users.insert(u0)
assert u.name=='jph'
users.insert(u1)
u

User(name='jph', pwd='foo')

In [ ]:
#| export
@patch
def __call__(
    self:DBTable,
    where:str|None=None,  # SQL where fragment to use, for example `id > ?`
    where_args: Iterable|dict|NoneType=None, # Parameters to use with `where`; iterable for `id>?`, or dict for `id>:id`
    order_by: str|None=None, # Column or fragment of SQL to order by
    limit:int|None=None, # Number of rows to limit to
    offset:int|None=None, # SQL offset
    select:str = "*", # Comma-separated list of columns to select
    **kw  # Combined with `where_args`
)->list:
    "Result of `select` query on the table"
    if select == "*": query = sa.select(self.table)
    else:
        columns = [sa.text(col.strip()) for col in select.split(',')]
        query = sa.select(*columns).select_from(self.table)
    if where_args: kw = {**kw, **where_args}
    if kw: query = query.where(sa.text(where).bindparams(**kw))
    if order_by: query = query.order_by(sa.text(order_by))
    if limit is not None: query = query.limit(limit)
    if offset is not None: query = query.offset(offset)
    rows = self.conn.execute(query).all()
    return [self.cls(**row._asdict()) for row in rows]

In [ ]:
assert users()==[u0,u1]
users()

[User(name='jph', pwd='foo'), User(name='rlt', pwd='bar')]

In [ ]:
r = users(where="pwd LIKE :pwd", pwd="b%")
assert r==[u1]
r

[User(name='rlt', pwd='bar')]

In [ ]:
assert len(todos())==1
todos()

[Todo(title='do it', name='jph', id=1, done=False, details='')]

In [ ]:
#| export
@patch
def _pk_where(self:DBTable, meth,key):
    if not isinstance(key,tuple): key = (key,)
    pkv = zip(self.table.primary_key.columns, key)
    cond = sa.and_(*[col==val for col,val in pkv])
    return getattr(self.table,meth)().where(cond)

In [ ]:
#| export
class NotFoundError(Exception): pass

In [ ]:
#| export
@patch
def __getitem__(self:DBTable, key):
    "Get item with PK `key`"
    qry = self._pk_where('select', key)
    result = self.conn.execute(qry).first()
    if not result: raise NotFoundError()
    return self.cls(**result._asdict())

In [ ]:
assert users['jph']==u0
users['jph']

User(name='jph', pwd='foo')

In [ ]:
#| export
@patch
def update(self:DBTable, obj):
    d = _wanted(obj)
    pks = tuple(d[k.name] for k in self.table.primary_key)
    qry = self._pk_where('update', pks).values(**d).returning(*self.table.columns)
    result = self.conn.execute(qry)
    row = result.one()
    self.conn.commit()
    return self.cls(**row._asdict())

In [ ]:
u.pwd = 'new'
users.update(u)
users()

[User(name='jph', pwd='new'), User(name='rlt', pwd='bar')]

In [ ]:
#| export
@patch
def delete(self:DBTable, key):
    "Delete item with PK `key` and return count deleted"
    result = self.conn.execute(self._pk_where('delete', key))
    self.conn.commit()
    return result.rowcount

In [ ]:
assert users.delete('jph')
test_fail(lambda: users['jph'])

## SQLAlchemy helpers

In [ ]:
#| export
from fastcore.net import urlsave

from collections import namedtuple
from sqlalchemy import create_engine,text,MetaData,Table,Column,engine,sql
from sqlalchemy.sql.base import ReadOnlyColumnCollection
from sqlalchemy.engine.base import Connection
from sqlalchemy.engine.cursor import CursorResult

In [ ]:
#| export
@patch
def __dir__(self:MetaData): return self._orig___dir__() + list(self.tables)

@patch
def __dir__(self:ReadOnlyColumnCollection): return self._orig___dir__() + self.keys()

def _getattr_(self, n):
    if n[0]=='_': raise AttributeError
    if n in self.tables: return self.tables[n]
    raise AttributeError

MetaData.__getattr__ = _getattr_

In [ ]:
dbm = db.meta

In [ ]:
' '.join(dbm.tables)

'user todo'

In [ ]:
t = dbm.todo

In [ ]:
list(t.c)

[Column('title', String(), table=<todo>),
 Column('name', String(), table=<todo>),
 Column('id', Integer(), table=<todo>, primary_key=True, nullable=False),
 Column('done', Boolean(), table=<todo>),
 Column('details', String(), table=<todo>)]

In [ ]:
#| export
@patch
def tuples(self:CursorResult, nm='Row'):
    "Get all results as named tuples"
    rs = self.mappings().fetchall()
    nt = namedtuple(nm, self.keys())
    return [nt(**o) for o in rs]

@patch
def sql(self:Connection, statement, nm='Row', *args, **kwargs):
    "Execute `statement` string and return results (if any)"
    if isinstance(statement,str): statement=text(statement)
    t = self.execute(statement)
    return t.tuples()

@patch
def sql(self:MetaData, statement, *args, **kwargs):
    "Execute `statement` string and return `DataFrame` of results (if any)"
    return self.conn.sql(statement, *args, **kwargs)

In [ ]:
rs = dbm.sql('select * from user')
rs[0]

Row(name='rlt', pwd='bar')

In [ ]:
#| export
@patch
def get(self:Table, where=None, limit=None):
    "Select from table, optionally limited by `where` and `limit` clauses"
    return self.metadata.conn.sql(self.select().where(where).limit(limit))

In [ ]:
t.get(t.c.title.startswith('d'), limit=5)

[Row(title='do it', name='jph', id=1, done=False, details='')]

This is the query that will run behind the scenes:

In [ ]:
print(t.select().where(t.c.title.startswith('d')).limit(5))

SELECT todo.title, todo.name, todo.id, todo.done, todo.details 
FROM todo 
WHERE (todo.title LIKE :title_1 || '%')
 LIMIT :param_1


In [ ]:
#| export
@patch
def close(self:MetaData):
    "Close the connection"
    self.conn.close()

In [ ]:
dbm.close()

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()